In [1]:
from numba import jit
'''
Implements the MFROLS algorithm (see page 97 from Billings, SA (2013)).
% written by: Renato Naville Watanabe 
% beta = mfrols(p, y, pho, s)
% Inputs:
%   p: matrix of floats, is the matrix of candidate terms.
%   y: vector of floats, output signal.
%   pho: float, stop criteria.
%   s: integer, iteration step of the mfrols algorithm.
% Output:
%   beta: vector of floats, coefficients of the chosen terms.
% Globals:
%   l: vector of integers, indices of the chosen terms.
%   err: vector of floats, the error reduction ratio of each chosen term.
%   ESR: float, the sum of the individual error reduction ratios.
%   A: matrix of floats, auxiliary matrix in the orthogonalization process. 
%   q: matrix of floats, matrix with each column being the terms orthogonalized by the Gram-Schmidt process.
%   g: vector of floats, auxiliary vector in the orthogonalization process.
%   M0:	integer, number of chosen terms.
'''
#@jit
def mfrols(p, y, pho, s):
    import numpy as np
    ## global variables assignment    
    global l
    global err
    global ESR
    global A
    global q 
    global g 
    global M0
    
    if np.ndim(p) == 2:
        pTemp = np.zeros((np.shape(p)[0],np.shape(p)[1],1))
        pTemp[:,:,0] = p
        p = pTemp
    if np.ndim(y) == 1:
        yTemp = np.zeros((np.shape(y)[0],1))
        yTemp[:,0] = y
        y = yTemp
        print(y)
    if s == 0:
        M = np.shape(p)[1]
        l = -1*np.ones((M))
        err = np.zeros((M))
        A = np.empty((M,M,1))
        q = np.empty_like(p)
        g = np.empty((1,M))


    M = np.shape(p)[1]
    L = np.shape(p)[2]
    gs= np.zeros((L,M))
    ERR=np.zeros((L,M))
    qs=np.zeros_like(p)

    for j in range(L):
        sigma = np.transpose(y[:,j])@y[:,j]
        for m in range(M):
            if np.max(m*np.ones_like(l)==l)==0:
                ## The Gram-Schmidt method was implemented in a modified way, as shown in Rice, JR(1966)
                qs[:,m,j] = p[:,m,j]
                for r in range(s):
                    qs[:,m,j] = qs[:,m,j] - (np.transpose(np.squeeze(q[:,r,j]))@qs[:,m,j])/(np.transpose(np.squeeze(q[:,r,j]))@np.squeeze(q[:,r,j]))*np.squeeze(q[:,r,j])
                gs[j,m] = (np.transpose(y[:,j])@np.squeeze(qs[:,m,j]))/(np.transpose(np.squeeze(qs[:,m,j]))@np.squeeze(qs[:,m,j]))
                ERR[j,m] = (gs[j,m]**2)*(np.transpose(np.squeeze(qs[:,m,j]))@np.squeeze(qs[:,m,j]))/sigma
            else:
                ERR[j,m]=0   

    
    ERR_m = np.mean(ERR, 0)
    l[s] = np.nonzero(ERR_m == np.max(ERR_m))[0]
    err[s] = ERR_m[int(l[s])]
    for j in range(L):
        for r in  range(s-1):
            A[r, s, j] = (np.transpose(q[:,r,j])@p[:,int(l[s]),j])/(np.transpose(q[:,r,j])@q[:,r,j])    
        A[s, s, j] = 1
        q[:, s,j] = qs[:,int(l[s]),j]
        g[j,s] = gs[j,int(l[s])]    

    ESR = ESR - err[s]   

    ## recursive call 

    if (err[s] >= pho and s < M-1):
        s += 1
        del qs 
        del gs
        beta = mfrols(p, y, pho, s)
    else:
        s += 1  
        M0 = s              
        beta = np.empty((M0,L))
        for j in range(L):
            if s > 1:
                beta[:,j] = np.linalg.inv(np.squeeze(A[0:M0,0:M0,j]))@np.transpose(g[j,0:M0])
            else:
                beta[:,j] = (np.squeeze(A[0:M0,0:M0,j])**-1)*g[j,0:M0]
    return beta  


In [2]:
import numpy as np
import time

In [3]:
N = 5000

In [4]:
x = np.random.randn(N)

In [5]:
y = np.empty((N,1))
y[:,0] = 10.5*x**3+2*x+10
D = dict()
D[0] = '1'
D[1] = 'x'
D[2] = 'x^2'
D[3] = 'x^3'
D

{0: '1', 1: 'x', 2: 'x^2', 3: 'x^3'}

In [6]:
p = np.empty((N,4,1))
p[:,:,0] = np.transpose(np.vstack((np.ones_like(x),x,x**2,x**3)))


In [7]:
s = 0
ESR = 1


In [8]:
tic = time.time()
beta = mfrols(p, y, 0.01,s)
toc = time.time()
print(toc-tic)

0.15406155586242676


In [9]:
print(beta)
D1 = dict()
for i in range(np.size(beta)):
    D1[i] = D[int(l[i])]
D1

[[ 10.42606079]
 [ 10.01490804]
 [  2.        ]]


{0: 'x^3', 1: '1', 2: 'x'}

In [ ]:
1800+567.51+133.6

In [10]:
"""
Created on Mon Nov 27 12:04:11 2017

@author: maysa
"""
#import mfrols
import numpy as np
p=np.array([[1.5,0,0],[0, -2,0],[0,0,2],[0,6,0],[0,3,0],[0,-4,0],[-2.5,0,4],[0,10,0],[0,20,0]])
y=np.array([[1],[0],[1],[0],[0],[0],[1],[0],[0]])
s = 0
ESR = 1
pho = 0.5
beta = mfrols(p,y,pho,s)
print(beta)

[[ 0.3       ]
 [ 0.57142857]]


In [11]:
s = 0
ESR = 1
beta = mfrols(p,y,pho,s)

In [12]:
p

array([[  1.5,   0. ,   0. ],
       [  0. ,  -2. ,   0. ],
       [  0. ,   0. ,   2. ],
       [  0. ,   6. ,   0. ],
       [  0. ,   3. ,   0. ],
       [  0. ,  -4. ,   0. ],
       [ -2.5,   0. ,   4. ],
       [  0. ,  10. ,   0. ],
       [  0. ,  20. ,   0. ]])

In [13]:
np.ndim(p)

2

In [ ]:
s

In [ ]:
l

In [ ]:
beta

In [ ]:
p